In [ ]:
%pip install ipywidgets

In [10]:


import ipywidgets as widgets
w = widgets.IntSlider()


file = open("test_image.png", "rb")
image = file.read()
widgets.Image(
    value = image,
    format = 'png',
    width = 1000,
    height = 1000              
    )



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xe8\x00\x00\x03\xe8\x08\x02\x00\x00\x00\xc2\xc1C…